In [ ]:
import ee
import pandas as pd
from datetime import datetime
# Initialize the Earth Engine module
ee.Authenticate()
ee.Initialize(project='osprey')

import ast

In [ ]:
# Define your Area of Interest (AOI)
#AOI = ee.Geometry.Rectangle([91.5847, 26.0934, 91.7847, 26.2934])  # Replace with your coordinates
AOI = ee.FeatureCollection('projects/osprey/assets/Megathon_osprey_flood_file')
# Function to calculate RFDI and other features

def rfdi_fun(img):
   # Convert to linear units (power)
   pwr = ee.Image(10).pow(img.divide(10)).set('system:time_start', img.get('system:time_start'))

   # Calculate RFDI (Radar Forest Disturbance Index)
   rfdi = pwr.select('VV').subtract(pwr.select('VH')).divide(pwr.select('VV').add(pwr.select('VH'))).rename('RFDI')

   # Calculate VV/VH ratio
   VV_VH = img.select('VV').divide(img.select('VH')).rename('VV_VH')

   # Calculate RVI (Radar Vegetation Index)
   rvi = pwr.expression('4*VH/(VV+VH)', {'VH': pwr.select('VH'), 'VV': pwr.select('VV')}).rename('RVI')

   # Add the calculated bands to the image
   return img.addBands(rfdi).addBands(VV_VH).addBands(rvi)

# Create a list to store statistics for each time period
stats_list = []

# Set the initial start and end dates
start_date = ee.Date('2022-01-01')
end_date = ee.Date('2022-12-10')

# Define the total duration you want to loop over (in days)
total_days = 343  # Change as needed
num_intervals = total_days // 7

# Loop through the defined number of intervals
for i in range(num_intervals + 1):
    S1start_date = start_date.advance(i * 7, 'days')
    S1end_date = S1start_date.advance(7, 'days')  # 7 days interval

    # Fetch the Sentinel-1 ImageCollection for the current period
    s1_All = (
        ee.ImageCollection('COPERNICUS/S1_GRD')
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
        .filter(ee.Filter.eq('instrumentMode', 'IW'))
        .filter(ee.Filter.inList('orbitProperties_pass', ['ASCENDING', 'DESCENDING']))
        .filterDate(S1start_date, S1end_date)
        .filterBounds(AOI)
    )
    # Create a composite and apply a filter to reduce noise
    S1All = s1_All.map(rfdi_fun).median()
    S1_C = ee.Image.cat([S1All.select(['VV', 'VH', 'RFDI'])]).clip(AOI).focalMean(30, 'square', 'meters')

    # Calculate features if there's data
    if s1_All.size().getInfo() > 0:
        s1_image = s1_All.first()
        vv = s1_image.select('VV')
        vh = s1_image.select('VH')
        vv_vh_ratio = vv.divide(vh).rename('VV_VH_ratio')
        s1_features = ee.Image.cat([vv, vh, vv_vh_ratio])

        # Calculate statistics
        stats = s1_features.reduceRegion(reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True), geometry=AOI, scale=10, maxPixels=1e9).getInfo()

        # Store the stats along with the start and end dates
        stats_list.append({
            'start_date': S1start_date.getInfo(),
            'end_date': S1end_date.getInfo(),
            'VV_mean': stats['VV_mean'],
            'VV_stdDev': stats['VV_stdDev'],
            'VH_mean': stats['VH_mean'],
            'VH_stdDev': stats['VH_stdDev'],
            'VV_VH_ratio_mean': stats['VV_VH_ratio_mean'],
            'VV_VH_ratio_stdDev': stats['VV_VH_ratio_stdDev'],
        })

# Convert the stats list to a DataFrame and save as CSV
stats_df = pd.DataFrame(stats_list)
stats_df.to_csv('sentinel_1_stats.csv', index=False)


print("Statistics saved to sentinel_1_stats.csv")


In [ ]:


def convert_timestamp(timestamp_str):
    # Safely evaluate the string as a dictionary using ast.literal_eval
    timestamp_dict = ast.literal_eval(timestamp_str)
    timestamp_millis = timestamp_dict['value']
    return datetime.utcfromtimestamp(timestamp_millis / 1000).strftime('%Y-%m-%d %H:%M:%S')


# Load the CSV file
input_file = '/content/sentinel_1_stats.csv'
output_file = '/content/sentinel_1_stats_final.csv'

# Read the CSV file
df = pd.read_csv(input_file)

# Convert 'start_date' and 'end_date' columns
df['Start'] = df['start_date'].apply(convert_timestamp)
df['end'] = df['end_date'].apply(convert_timestamp)

# Save the updated DataFrame to a new CSV file
df.to_csv(output_file, index=False)

print(f"Converted dates saved to {output_file}")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import joblib  # for saving the model

# Load the data
data = pd.read_csv('/content/sentinel_1_stats_assam_riverside.csv')

# Separate features and target variable
X = data.drop(columns=['flood','Start', 'end'])
y = data['flood']

# Perform stratified 80:20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, stratify=y, random_state=42)

# Initialize and train the logistic regression model
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Save the model parameters
joblib.dump(log_reg, '/content/logistic_regression_model.pkl')

# Predict on the test set
y_pred = log_reg.predict(X_test)
y1_pred = log_reg.predict(X_train)
# Evaluate the model
train_accuracy = accuracy_score(y_train, y1_pred)
test_accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'train Accuracy: {train_accuracy}')
print(f'test Accuracy: {test_accuracy}')
print('Classification Report:')
print(report)


train Accuracy: 0.8833333333333333
test Accuracy: 0.8785714285714286
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.95      0.91        91
           1       0.88      0.76      0.81        49

    accuracy                           0.88       140
   macro avg       0.88      0.85      0.86       140
weighted avg       0.88      0.88      0.88       140



app ke liye

In [ ]:
# Define your Area of Interest (AOI)
AOI = ee.Geometry.Rectangle([91.5847, 26.0934, 91.7847, 26.2934])  # Replace with your coordinates
# Function to calculate RFDI and other features

def rfdi_fun(img):
   # Convert to linear units (power)
   pwr = ee.Image(10).pow(img.divide(10)).set('system:time_start', img.get('system:time_start'))

   # Calculate RFDI (Radar Forest Disturbance Index)
   rfdi = pwr.select('VV').subtract(pwr.select('VH')).divide(pwr.select('VV').add(pwr.select('VH'))).rename('RFDI')

   # Calculate VV/VH ratio
   VV_VH = img.select('VV').divide(img.select('VH')).rename('VV_VH')

   # Calculate RVI (Radar Vegetation Index)
   rvi = pwr.expression('4*VH/(VV+VH)', {'VH': pwr.select('VH'), 'VV': pwr.select('VV')}).rename('RVI')

   # Add the calculated bands to the image
   return img.addBands(rfdi).addBands(VV_VH).addBands(rvi)

# Create a list to store statistics for each time period
stats_list = []

# Set the initial start and end dates
start_date = ee.Date('2022-01-01') #yeh date hoga, jo current date hai uss se 14 din pahle ka, and then make s
end_date = ee.Date('2022-12-10') #current date

# Define the total duration you want to loop over (in days)
total_days = 14 # Change as needed
num_intervals = total_days // 7
s1_All = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.inList('orbitProperties_pass', ['ASCENDING', 'DESCENDING']))
    .filterDate(S1start_date, S1end_date)
    .filterBounds(AOI)
)
    # Create a composite and apply a filter to reduce noise
S1All = s1_All.map(rfdi_fun).median()
S1_C = ee.Image.cat([S1All.select(['VV', 'VH', 'RFDI'])]).clip(AOI).focalMean(30, 'square', 'meters')

    # Calculate features if there's data
if s1_All.size().getInfo() > 0:
    s1_image = s1_All.first()
    vv = s1_image.select('VV')
    vh = s1_image.select('VH')
    vv_vh_ratio = vv.divide(vh).rename('VV_VH_ratio')
    s1_features = ee.Image.cat([vv, vh, vv_vh_ratio])

        # Calculate statistics
    stats = s1_features.reduceRegion(reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True), geometry=AOI, scale=10, maxPixels=1e9).getInfo()

        # Store the stats along with the start and end dates
    stats_list.append({
        'start_date': S1start_date.getInfo(),
        'end_date': S1end_date.getInfo(),
        'VV_mean': stats['VV_mean'],
        'VV_stdDev': stats['VV_stdDev'],
        'VH_mean': stats['VH_mean'],
        'VH_stdDev': stats['VH_stdDev'],
        'VV_VH_ratio_mean': stats['VV_VH_ratio_mean'],
        'VV_VH_ratio_stdDev': stats['VV_VH_ratio_stdDev'],
    })

# Convert the stats list to a DataFrame and save as CSV
stats_df = pd.DataFrame(stats_list)
stats_df.to_csv('sentinel_1_stats.csv', index=False)


In [ ]:
from sklearn.metrics import classification_report, accuracy_score
import joblib
import pandas as pd
path = "path of the test file generated for a particular user"
def predict(model_path, test_data_path):
    # Load the model
    model = joblib.load(model_path)

    # Load the test data
    data = pd.read_csv(test_data_path)
    X_test = data.drop(columns=['','Start', 'end'])
    y_test = data['flood']

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Evaluate the model
    print(y_pred)

# Call the function
predict('/content/logistic_regression_model.pkl', path)
